## Chiptune Generation Project Sandbox

Notebook for trying out some code in an interactive, sandbox-like environment.

In [2]:
from modules import mcc_parser, mcc_markov, mcc_waves, mcc_builder

import IPython.display as ipd
import matplotlib.pyplot as plt
%matplotlib inline

sr = 44100

In [3]:
"""
Single track example.
"""

# Open the MidiFile.
mid = mcc_parser.open_midi("data/Zgbreve.mid")
# Extract attributes such as tempo and time signature from header track.
info = mcc_parser.extract_midi_info(mid.tracks[0])
tracks = mcc_parser.extract_midi_tracks(mid.tracks)

# First track.
track = mcc_parser.midi_to_rtttl(tracks[0], mid.ticks_per_beat)

# Train and predict.
mm = mcc_markov.KMarkov(3)
mm.fit(track)
# Passing the track we just trained on allows us to control the starting note of prediction.
gen = mm.predict(samples=50, priors=track)

# Join the original notes with the generated notes.
full = mcc_builder.join(track, ",", gen)

# Render the notes as a waveform.
res = mcc_waves.notes_to_waveform(full, bpm=info["tempo"][0], wave_function=mcc_waves.triangle_wave)

In [5]:
# ipd.Audio(res, rate=sr)

In [4]:
res = mcc_waves.notes_to_waveform(track, bpm=info["tempo"][0], wave_function=mcc_waves.triangle_wave)
# ipd.Audio(res, rate=sr)

In [9]:
"""
Multi-track example.
"""

# Open the MidiFile.
mid = mcc_parser.open_midi("data/Zgbreve.mid")
# Extract attributes such as tempo and time signature from header track.
info = mcc_parser.extract_midi_info(mid.tracks[0])
tracks = mcc_parser.extract_midi_tracks(mid.tracks)

out = []

for track in tracks:
    track = mcc_parser.midi_to_rtttl(track, mid.ticks_per_beat)

    # Train and predict.
    mm = mcc_markov.KMarkov(30)
    mm.fit(track)
    gen = mm.predict(50)

    # Join the original notes with the generated notes.
    full = mcc_builder.join(track, ",", gen)
    
    # Render the notes as a waveform.
    res = mcc_waves.notes_to_waveform(full, bpm=info["tempo"][0], wave_function=mcc_waves.triangle_wave)
    out.append(res)

# TODO: this track combination does not line up the 
# tracks correctly. Results in tracks out of sync.
done = mcc_builder.combine_tracks(out)


In [6]:
# ipd.Audio(done, rate=sr)